# **타이타닉 데이터를 통한 사망자 예측**
---

### **.csv파일을 불러오기**

In [8]:
import numpy as np  
import pandas as pd  

import os
for dirname, _, filenames in os.walk('./datafile'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


./datafile\gender_submission.csv
./datafile\test.csv
./datafile\train.csv


os.walk(path)를 사용할 경우 필요함
* dirname(root) : dir, file이 들어있는 경로
* _(dirs) : root 아래의 폴더들
* filenames(files) : root 아래의 파일들  

os.path.join(경로,파일) : 두개 join 시켜줌


In [18]:
train_data = pd.read_csv("./datafile/train.csv")
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


pd.read.csv(파일경로/파일명) : 파일의 자료를 보여줌  
pd.read.csv.head() : 파일의 자료 중 상위 5개 항목만 표시   
pd.read.csv.head(int) : 상위 n개 항목 표시

In [19]:
test_data = pd.read_csv("./datafile/test.csv") 
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### **남녀 생존 비율 구하기**

In [31]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived, rate_women :", rate_women)

% of women who survived, rate_women : 0.7420382165605095


첫 줄의 뜻은 train_data.Sex가 female인 자료들의 Survived 열을  
women에 리스트로 저장이라고 생각된다  
len(women)은 전체 행의 개수  
sum(women)은 Survived(0,1)에 따라 계산됨

In [32]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)
print("% of men who survived, rate_men :", rate_men)


% of men who survived, rate_men : 0.18890814558058924


### **머신 러닝 모델 만들기**

In [42]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex" , "SibSp" , "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X,y)
predictions = model.predict(X_test)

output = pd.DataFrame(
    {'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('./datafile/submission.csv', index=False)
print("Succeess")


Succeess


pd.get_dummies() : 
* 카테고리 데이터를 수치형으로 변환  
* Sex Column가 Sex_female과 Sex_male 두 개의 열로 나뉜 것을 알 수 있음  
* 즉 기존의 문자 데이터를 숫자형으로 바꾸어 나타내도록 만듬  

RandomForestClassifier() :
* 트리 만들기
* n_estimators : 사용할 트리 개수
* max_depth : 트리의 최대 깊이
* random_state : 난수 seed

.fit(x_train, y_train) : (아직 정확히 모르겠지만) 학습하는 함수  
.predict() : (아마도) 예측하는 함수
pd.Dataframe() : 데이터 프레임을 만드는 함수  
.to_csv : 데이터 프레임을 csv파일로 내보내기


# **랜덤 포레스트**
---


<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile22.uf.tistory.com%2Fimage%2F995B513C5C5FEE3A0FA8BD"></img>

* 흔히 스무고개를 하면 떠올릴만한 **이진형 답변**을 통해 점점 내려가며 트리를 형성한다.  
* 여러개의 무작위성을 지닌 트리를 많이 만들기에 **랜덤 포레스트**라고 한다.  
* 하나의 데이터에 대해 모든 트리가 같은 답변을 해주진 않을 것이다.  
* 이렇기에 각 트리들의 결과를 다수결을 통하여 진짜 결론을 내리는 것을 **앙상블**이라고 한다.  
* 이러한 랜덤포레스트에 가장 큰 영향을 미치는 매개변수는 **포레스트의 크기**(트리의 개수)와 **최대 허용깊이**이다.  
* 일반성을 높이기 위해 트리의 개수를 많이 생성하면 트리의 개수가 많을수록 정확성을 올라가지만 학습시간이 길어지게된다.  
* 최대 허용 깊이는 맨 위의 노드부터 가장 아래의 노드까지의 깊이를 설정하는 것이다.
